# <span style="color:#820747">1. WEB SCRAPING INDEED.COM 
                                                                                          by Pavel Tarasenko

# <span style="color:#6e750e">Navigator:

<b>[2. My EDA](./2_EDA.ipynb)

<b>[3. Salary Prediction](./3_MODELING_Part1.ipynb)

<b>[4. Factors that distinguish job category](./4_Factors that distinguish job category.ipynb)

<b>[5. TECHNICAL REPORT](./Technical_report.txt)

<b>[6. NON TECHNICAL REPORT](./Non_technical_report.txt)

In [1]:
# Import all necessary libraries.  #fe019a
# -----------------------------------------------------
import matplotlib.pyplot as plt
import pandas as pd
import bs4
import re
import time
import numpy as np
import sklearn
from time import sleep
import requests
import seaborn as sns

from statistics import median
import matplotlib as mpl
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request, urllib.parse, urllib.error

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec">Defining functions to get text after BeautifulSoup format the html file. When I will get  text I will create a DataFrame where we will place our picked text, so it will become our scraped data. I researched Indeed website to find right paths which I will use to get data from list where I will put lxml file.  

In [15]:
def get_job(result):
    try:
        return result.find('a', {'data-tn-element':'jobTitle'}).text
    except:
        return 'NA'
    
def get_comp(result):
    try:
        return result.find('span', {'class':'company'}).text
    except:
        return 'NA'
    
def get_loc(result):
    try:
        return result.find('span', {'class':'location'}).text
    except:
        return 'NA'
    
def get_sal(result):
    try:
        return result.find('span', {'class':'no-wrap'}).text
    except:
        return 'NA'
    
def get_summ(result):
    try:
        return result.find('span', {'class':'summary'}).text
    except:
        return 'NA'
    
def get_bio(bio):
    try:
        return bio.find('td', {'class' : "snip"}).text
    except:
        return 'NA'

def get_day(bio):
    try:
        return bio.find('span', {'class' : "date"}).text
    except:
        return 'NA'

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec">SET Cities and jobs titles, I will be using it in our loop to get data I am interesting in. '+' sign I use as that how url for this website works.

In [9]:
test_city = ['Sydney','Melbourne', 'Perth', 'Adelaide', 'Brisbane']
test_job = ['data+scientist', 'data+analyst', 'business+analyst', 'big+data']

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec">Set loop to scrape main page with jobs, and subpage with big summary about job position. I use only BeautifulSoup for this.

- for start in range(0, 1000, 10) - as max pages on website is 100 I set 1000 with step 10, as urs counts pages by 10
- html = requests.get(url) - geting responce from the url I'm intrestin in.
- soup = BeautifulSoup(html.text, 'lxml') - getting html and using BeautifulSoup to get lxml format.
- Then using 2 loops to get only text I'm interesting in with tags, and put it into lists for futhure func processing.

In [98]:
results = []
bio = []
for job in test_job: # ------------------------------------------------------- first loop to go through list of jobs
    for city in test_city: # ---------------------------------------------- second loop to go through list of cities
        for start in range(0, 1000, 10): # ---------- third loop to go through pages, range 0 - 1000 with step of 10
            url = "https://au.indeed.com/jobs?q=" + job + "&l=" + city + "&start=" + str(start) # url for main pages
            html = requests.get(url) # ------------------------- getting responce from url we are requesting our url
            soup = BeautifulSoup(html.text, 'lxml') # ------------------ use BS with lxml to parse our html from url        
            for result in soup.find_all('div', {'class':' row result'}): # - select only part we need from main page
                results.append(result)  # ----------------------------------- add selected part to the list 'result'
            for i in soup.find_all('div', {'class':' row result'}): # ---- this loop we will use to get subpage text
                a = i.find('a') # --------------------- looking for tag <a ... which lead to big summary for the job
                ab ='https://au.indeed.com/' +  a.attrs['href'] # ------------ set right address to big bio(summary)
                html2 = requests.get(ab)  # --------------------------------- getting responce from big bio sub page
                soup2 = BeautifulSoup(html2.text, 'lxml') # ------------ use BS with lxml to parse our html from url   
                bio.append(soup2)  # ------------------------------------------- add selected part to the list 'bio'

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec">Set column names and 2 DataFrames, 1st for main info and second for big summary, later I will concatenate them.

In [81]:
jobs555 = pd.DataFrame(columns=['location', 'title', 'company', 'salary', 'summary', 'day'])
jobs111 = pd.DataFrame(columns=['job_bio'])

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec"> First for loop I will use to define my variables using my function which I created before, and on the each iteration add to the DataFrame 'jobs555'. Second loop I do the same thing but only for the big summary.

In [82]:
for entry in results:
    location = get_loc(entry)
    title = get_job(entry)
    company = get_comp(entry)
    salary = get_sal(entry)
    summary = get_summ(entry)
    day = get_day(entry)
    jobs555.loc[len(jobs555)] = [location, title, company, salary, summary, day]
    
for entry2 in bio:
    job_bio = get_bio(entry2)
    jobs111.loc[len(jobs111)] = [job_bio]

# <span style="color:#820747">Cleaning my Scraped Data

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec"> At this stage my Data was scraped and placed in to two different DataFrames, now I will do some cleaning (removing unneeded characters, spaces, etc.). Also concatenate two DataFrames into one.

In [83]:
# Remove spaces. 
# -----------------------------------------------------
jobs1 = jobs111.replace(r'\n',' ', regex = True) 
jobs0 = jobs555.replace(r'\n',' ', regex = True) 

In [105]:
# Check scraped data shape. 26613 lines, there should be many duplicates
# -----------------------------------------------------
print(jobs0.shape)
print(jobs1.shape) 

(26613, 7)


In [135]:
# Save DataFrames in to csv files, use it as back up. Load this data into new variables 'jobs0' and 'jobs1'
# -----------------------------------------------------
# jobs0.to_csv('raw_1_part_secondedition.csv')
# jobs1.to_csv('raw_2_part_secondedition.csv')
jobs0 = pd.read_csv('raw_1_part_secondedition.csv')
jobs1 = pd.read_csv('raw_2_part_secondedition.csv')

In [137]:
# Concatenate 2 DataFrames by columns (axis = 1) and assign it in to new variable 'row_combo' 
# -----------------------------------------------------
row_combo = pd.concat([jobs0, jobs1], axis = 1)

In [140]:
# Check data for duplicates, her I have got 22097 lines of them. =( Also I check dublicates only for subset which
# is not included big summary as there is many strange simbols, what can cause count more duplicates but 
# they are not in real necessary to be duplicated
# -----------------------------------------------------
row_combo.duplicated(subset = ['location', 'title', 'company', 'salary', 'summary', 'day']).sum()

22097

In [142]:
# Dropping duplicates from subset and save changes 'inplace=True'
# -----------------------------------------------------
row_combo.drop_duplicates(subset = ['location', 'title', 'company', 'salary', 'summary', 'day'], inplace = True)

In [149]:
# I got 4479 lines with non duplicated data. 
# -----------------------------------------------------
row_combo.shape

(4479, 9)

In [132]:
# Using lambda function I change all strings 'NA' to numeric 0 'np.nan'
# -----------------------------------------------------
row_combo.location = row_combo.location.map(lambda x: np.nan if x == 'NA' else str(x))
row_combo.title = row_combo.title.map(lambda x: np.nan if x == 'NA' else str(x))
row_combo.company = row_combo.company.map(lambda x: np.nan if x == 'NA' else str(x))
row_combo.salary = row_combo.salary.map(lambda x: np.nan if x == 'NA' else str(x))
row_combo.summary = row_combo.summary.map(lambda x: np.nan if x == 'NA' else str(x))
row_combo.job_bio = row_combo.job_bio.map(lambda x: np.nan if x == 'NA' else str(x))

In [147]:
# Now I drop all 0 values but no included 'salary' as I will need it, later for my prediction. 
# I drop lines here not columns (axis = 0)
# -----------------------------------------------------
row_combo.dropna(subset = ['location', 'title', 'company', 'summary', 'day', 'job_bio'], axis = 0, inplace = True)

In [148]:
# Now I can see that our data does not have any nulls except 'salary', that what I need.
# -----------------------------------------------------
row_combo.isnull().sum()

Unnamed: 0       0
location         0
title            0
company          0
salary        3605
summary          0
day              0
Unnamed: 0       0
job_bio          0
dtype: int64

In [151]:
# Delete all spaces using map function with 'str.strip' in dataset, as we can see spaces in csv file and cant use 
# our data for future analysis. We cant using strip do it for salary as it included NaN values. 
# -----------------------------------------------------
row_combo["summary"] = row_combo["summary"].map(str.strip)
row_combo["location"] = row_combo["location"].map(str.strip)
row_combo["title"] = row_combo["title"].map(str.strip)
row_combo["company"] = row_combo["company"].map(str.strip)
row_combo["day"] = row_combo["day"].map(str.strip)
row_combo["job_bio"] = row_combo["job_bio"].map(str.strip)

In [228]:
# Saving data for back up and loading this data to new variable 'df'
# -----------------------------------------------------
#row_combo.to_csv('Mega_new.csv')
df = pd.read_csv('Mega_new.csv')

In [230]:
# Check our DataFrame, here we can see Unnamed columns as I didnt set index = False when saving my file. 
# -----------------------------------------------------
df.head(1)

,Unnamed: 0.1,location,title,company,salary,summary,day,Unnamed: 0.2,job_bio
0,0,Sydney NSW,Junior Data Analyst/Scientist,International Institute of Data & Analytics,NaN,"In data science and big data analytics, the ID...",1 day ago,0,The International Institute of Data & Analytic...


In [ ]:
# Remove unneeded columns. 
# -----------------------------------------------------
df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'] , axis=1, inplace=True)

# <span style="color:#820747">Start cleaning salary column.

<span style="color:red">
--- Comment ---

<span style="color:#1d5dec"> I will make salary column with an annual salary format. Also I will remove all characters I dont need. I will create new collumn where i will keep type of salary which was before reformating into annual format.

In [231]:
# Remove spaces, commas and dollar sign from all salary values. 
# -----------------------------------------------------
df['salary'] = df['salary'].str.replace('\n', '')
df['salary'] = df['salary'].str.replace(',', '')
df['salary'] = df['salary'].str.replace('$', '')

In [232]:
# Use map function and lambda to bring all values in salary into string format for future formating.
# -----------------------------------------------------
df.salary = df.salary.map(lambda x: str(x))

In [233]:
# Create column 'Salary_type' and if the salary contains information about payment type such as 
# (month, year, day, week) then assign that values to the new column.
# -----------------------------------------------------
df['Salary_type'] = np.nan
df.ix[df['salary'].str.contains('year'), 'Salary_type'] = 'year'
df.ix[df['salary'].str.contains('month'), 'Salary_type'] = 'month'
df.ix[df['salary'].str.contains('week'), 'Salary_type'] = 'week'
df.ix[df['salary'].str.contains('day'), 'Salary_type'] = 'day'
df.ix[df['salary'].str.contains('hour'), 'Salary_type'] = 'hour'

In [234]:
# Separating out data based on whether contains hour, day, week, month, year
# -----------------------------------------------------
year_salaries = df[df['salary'].str.contains('year')]
month_salaries = df[df['salary'].str.contains('month')]
week_salaries = df[df['salary'].str.contains('week')]
day_salaries = df[df['salary'].str.contains('day')]
hour_salaries = df[df['salary'].str.contains('hour')]
nan_salaries = df[df['salary'].str.contains('nan')]

In [3]:
# Removing string values(“ a year”, “ a week”, etc. from salary dfs), as all salary values has one format its easy.
# -----------------------------------------------------
year_salaries['salary'] = year_salaries['salary'].str.replace(' a year', '')
month_salaries['salary'] = month_salaries['salary'].str.replace(' a month', '')
week_salaries['salary'] = week_salaries['salary'].str.replace(' a week', '')
day_salaries['salary'] = day_salaries['salary'].str.replace(' a day', '')
hour_salaries['salary'] = hour_salaries['salary'].str.replace(' an hour', '')

In [236]:
# Defing the function which will split numbers from the salary and find avarage, if single value just 
# take it as it is.
# -----------------------------------------------------
def get_right_salary(x):
    try:
        spliter = x.split(' - ', 1)
        value_one = float(spliter[0])
        value_two = float(spliter[1])
        return (value_one + value_two) / 2  # return mean of those two values
    except:
        return float(x)  # if we cant split it means there is only one value, I am taking as it is in float format

In [4]:
# Apply our function to the salary column, and bring it in to the same format (year)
# -----------------------------------------------------
year_salaries['salary'] = year_salaries['salary'].apply(get_right_salary)
month_salaries['salary'] = month_salaries['salary'].apply(get_right_salary)
month_salaries['salary'] = month_salaries['salary'] * 12
week_salaries['salary'] = week_salaries['salary'].apply(get_right_salary)
week_salaries['salary'] = week_salaries['salary'] * 52
day_salaries['salary'] = day_salaries['salary'].apply(get_right_salary)
day_salaries['salary'] = day_salaries['salary'] * 260
hour_salaries['salary'] = hour_salaries['salary'].apply(get_right_salary)
hour_salaries['salary'] = hour_salaries['salary'] * 2080

In [239]:
# Rejoining salary data into combined_salaries, and assign all coluns to our new DataFrame (df)
# -----------------------------------------------------
combined_salaries = pd.concat([year_salaries, month_salaries, week_salaries, day_salaries, hour_salaries, nan_salaries], axis=0)
df = combined_salaries[['location', 'title', 'company', 'salary', 'Salary_type', 'summary', 'job_bio']]

<span style="color:red">
--- comment ---

<span style="color:#1d5dec">
In this table we can see our salary in the same format annualy and "salary type" column is related to type of payment was before we put salary in annual format. So later I will keep in mind that normally higher salary per anum related to salary which was for example per hour, which is means it was most likely contract job. 

In [242]:
# Salary_type shows in what format salary was before reformating
df.head(40)

,location,title,company,salary,Salary_type,summary,job_bio
1,Macquarie University NSW,Data Science Research Engineer,Macquarie University,106382,year,We are seeking a Data Science Research Enginee...,Macquarie is the university of pioneering mind...
18,Sydney NSW,Senior Software Engineer (Machine Learning),ROKT,137500,year,A strong statistical background and/or experie...,Exceptional opportunity to join one of the fas...
24,Sydney NSW,Event Research Analyst,Destination NSW-Corporate Services,93704.5,year,Outline your experience in carrying out market...,"Event Research Analyst - 0000650N Ongoing, Fu..."
31,Sydney NSW,Junior Data Scientist,Correlate Resources,75000,year,As a Junior Data Scientist the responsibilitie...,Our client is an industry pioneering Customer ...
36,Sydney NSW,Scientist,Science Division,108792,year,"Data layers, satellite imagery, photography, o...",Scientist - 00006173 Job Title: Scientist Job...
38,Sydney NSW,Data Scientist - Engineering,Onset Group Pty Ltd,145000,year,Big Data Technologies:. This is a significant ...,The core product integrates with external clie...
44,Sydney NSW,Data Scientist (Python/R),Correlate Resources,110000,year,Work within a team of industry leading Data Sc...,Our client is an industry pioneering Customer ...
46,Sydney NSW,Lead Data Scientist,7 Recruitment,200000,year,"Big data, data mining, machine learning, AI. G...",We are currently working with an organisation ...
49,Sydney NSW,Senior Full Stack Developer,CSIRO,118500,year,"Join CSIRO Data61's team, the largest data inn...",Are you looking for a new challenge? Ship rea...
53,Sydney Central Business District NSW,Senior Data Scientist - Leadership Position,Morgan McKinley,170000,year,The time has come to step up and be counted as...,Calling all Senior Data Scientists!!!! The ti...


In [243]:
# Check for duplicates. 
# -----------------------------------------------------
df.duplicated(subset=['location', 'title', 'company', 'salary', 'summary']).sum()

41

In [5]:
# After reformating we got few more duplicates which I will drop, again using only subset. 
# -----------------------------------------------------
df.drop_duplicates(subset=['location', 'title', 'company', 'salary', 'summary'], inplace=True)

In [245]:
# Shape of our DataFrame now 4438 lines and 7 columns. 
# -----------------------------------------------------
df.shape

(4438, 7)

In [247]:
# Save my DataFrame into new csv file for back up. 
# -----------------------------------------------------
df.to_csv('1_Mega_new_after salary.csv')

# <span style="color:#820747">Cleaning rest of the data

In [248]:
# Creating data character cleaning function, and applying to all columns, also lowercasing all string data for 
# easier to do nlp later. 
# -----------------------------------------------------
def prep_for_nlp(df, column):
    cleaning_list = ['+', '$','/',',','?','.',';','-','@','!','&','%','^','*',')','(','|','<', '>', '"', ':', u'‚',u'Ä',u'ô', u'ì' '\n']
    for item in cleaning_list:
        df[column] = df[column].str.replace(item, ' ')   
        df[column] = df[column].map(str.lower)

In [2]:
# Apply my functions to each column in dataset.
# -----------------------------------------------------
prep_for_nlp(df, 'job_bio')
prep_for_nlp(df, 'summary')
prep_for_nlp(df, 'company')                             
prep_for_nlp(df, 'title')   
prep_for_nlp(df, 'location') 

<span style="color:red">
--- comment ---

<span style="color:#1d5dec">
I can see that my data is pretty clean now, and I want to make location more cleaner. To do so, I will create 
functions which will check for key words and replace whole text in a cell to City related to that key words. 

In [250]:
df.head(2)

,location,title,company,salary,Salary_type,summary,job_bio
1,macquarie university nsw,data science research engineer,macquarie university,106382,year,we are seeking a data science research enginee...,macquarie is the university of pioneering mind...
18,sydney nsw,senior software engineer machine learning,rokt,137500,year,a strong statistical background and or experie...,exceptional opportunity to join one of the fas...


In [251]:
# Defing the functions which I can use to get city names from information given in location. 
# -----------------------------------------------------
def clean_loc1(x):
    if 'nsw' in  x:
        x = 'sydney'
    else:
        x = x
    return x

def clean_loc2(x):
    if 'vic' in  x:
        x = 'melbourne'
    else:
        x = x
    return x

def clean_loc3(x):
    if 'qld' in  x:
        x = 'brisbane'
    else:
        x = x
    return x

def clean_loc4(x):
    if 'wa' in  x:
        x = 'perth'
    else:
        x = x
    return x

def clean_loc5(x):
    if 'sa' in  x:
        x = 'adelaide'
    else:
        x = x
    return x

In [6]:
# Apply my unction to get cities. 
# -----------------------------------------------------
df.location = df.location.map(clean_loc1)
df.location = df.location.map(clean_loc2)
df.location = df.location.map(clean_loc3)
df.location = df.location.map(clean_loc4)
df.location = df.location.map(clean_loc5)

In [253]:
# Checking how my location looks like now. 
# -----------------------------------------------------
df.head(5)

,location,title,company,salary,Salary_type,summary,job_bio
1,sydney,data science research engineer,macquarie university,106382,year,we are seeking a data science research enginee...,macquarie is the university of pioneering mind...
18,sydney,senior software engineer machine learning,rokt,137500,year,a strong statistical background and or experie...,exceptional opportunity to join one of the fas...
24,sydney,event research analyst,destination nsw corporate services,93704.5,year,outline your experience in carrying out market...,event research analyst 0000650n ongoing fu...
31,sydney,junior data scientist,correlate resources,75000,year,as a junior data scientist the responsibilitie...,our client is an industry pioneering customer ...
36,sydney,scientist,science division,108792,year,data layers satellite imagery photography o...,scientist 00006173 job title scientist job...


In [254]:
# Saving new csv file for back up. 
# -----------------------------------------------------
#df.to_csv('2_Mega_new_ready_for_nlp_secondedition.csv')

-- Comment --

<span style="color:#1d5dec">
I put jobs in to 3 groups (senior, junior, and middle) so we can analise data more easier, we can see what position related to what level. Also we can see salary diference in between this groups. 

In [7]:
# Separating junior, senior and medium level.
# -----------------------------------------------------
df['level'] = np.nan
df.ix[df['title'].str.contains('senior'), 'level'] = 'senior'
df.ix[df['title'].str.contains('lead'), 'level'] = 'senior'
df.ix[df['title'].str.contains('junior'), 'level'] = 'junior'

In [263]:
df.head(100)

,location,title,company,salary,Salary_type,summary,job_bio,level
1,sydney,data science research engineer,macquarie university,106382,year,we are seeking a data science research enginee...,macquarie is the university of pioneering mind...,middle
18,sydney,senior software engineer machine learning,rokt,137500,year,a strong statistical background and or experie...,exceptional opportunity to join one of the fas...,senior
24,sydney,event research analyst,destination nsw corporate services,93704.5,year,outline your experience in carrying out market...,event research analyst 0000650n ongoing fu...,middle
31,sydney,junior data scientist,correlate resources,75000,year,as a junior data scientist the responsibilitie...,our client is an industry pioneering customer ...,junior
36,sydney,scientist,science division,108792,year,data layers satellite imagery photography o...,scientist 00006173 job title scientist job...,middle
38,sydney,data scientist engineering,onset group pty ltd,145000,year,big data technologies this is a significant ...,the core product integrates with external clie...,middle
44,sydney,data scientist python r,correlate resources,110000,year,work within a team of industry leading data sc...,our client is an industry pioneering customer ...,middle
46,sydney,lead data scientist,7 recruitment,200000,year,big data data mining machine learning ai g...,we are currently working with an organisation ...,senior
49,sydney,senior full stack developer,csiro,118500,year,join csiro data61's team the largest data inn...,are you looking for a new challenge ship rea...,senior
53,sydney,senior data scientist leadership position,morgan mckinley,170000,year,the time has come to step up and be counted as...,calling all senior data scientists the ti...,senior


In [8]:
# Make middle group if that job not in senior and junior
# -----------------------------------------------------
df.level = df.level.map(lambda x: 'middle' if x not in ['senior', 'junior'] else x)

In [264]:
# Saving new csv for backup
# -----------------------------------------------------
#df.to_csv('3_FOR_NLP.csv', index=False) 

In [265]:
# Read my file into df variable
# -----------------------------------------------------
df = pd.read_csv('3_FOR_NLP.csv')

In [266]:
df

,location,title,company,salary,Salary_type,summary,job_bio,level
0,sydney,data science research engineer,macquarie university,106382.0,year,we are seeking a data science research enginee...,macquarie is the university of pioneering mind...,middle
1,sydney,senior software engineer machine learning,rokt,137500.0,year,a strong statistical background and or experie...,exceptional opportunity to join one of the fas...,senior
2,sydney,event research analyst,destination nsw corporate services,93704.5,year,outline your experience in carrying out market...,event research analyst 0000650n ongoing fu...,middle
3,sydney,junior data scientist,correlate resources,75000.0,year,as a junior data scientist the responsibilitie...,our client is an industry pioneering customer ...,junior
4,sydney,scientist,science division,108792.0,year,data layers satellite imagery photography o...,scientist 00006173 job title scientist job...,middle
5,sydney,data scientist engineering,onset group pty ltd,145000.0,year,big data technologies this is a significant ...,the core product integrates with external clie...,middle
6,sydney,data scientist python r,correlate resources,110000.0,year,work within a team of industry leading data sc...,our client is an industry pioneering customer ...,middle
7,sydney,lead data scientist,7 recruitment,200000.0,year,big data data mining machine learning ai g...,we are currently working with an organisation ...,senior
8,sydney,senior full stack developer,csiro,118500.0,year,join csiro data61's team the largest data inn...,are you looking for a new challenge ship rea...,senior
9,sydney,senior data scientist leadership position,morgan mckinley,170000.0,year,the time has come to step up and be counted as...,calling all senior data scientists the ti...,senior
